In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uab-the-hack-2024/grupos_cleaned.csv
/kaggle/input/uab-the-hack-2024/caracteristicas.csv
/kaggle/input/uab-the-hack-2024/ubicaciones_cleaned.csv
/kaggle/input/uab-the-hack-2024/calendario_grupos_merged.csv
/kaggle/input/uab-the-hack-2024/franjas_cuarto_hora.csv
/kaggle/input/uab-the-hack-2024/calendario_grupos_clean.csv
/kaggle/input/uab-the-hack-2024/franjas_media_hora.csv
/kaggle/input/uab-the-hack-2024/grupos.csv
/kaggle/input/uab-the-hack-2024/merged_caracteristicas_recursos.csv
/kaggle/input/uab-the-hack-2024/not_merged_caracteristicas_recursos.csv
/kaggle/input/uab-the-hack-2024/recursos_caracteristicas.csv
/kaggle/input/uab-the-hack-2024/calendario_grupos.csv
/kaggle/input/uab-the-hack-2024/ubicaciones.csv
/kaggle/input/uab-the-hack-2024/grupos_ubicacio_merged.csv


La tabla contiene 515 registros y 16 columnas. Estas son las principales columnas relevantes para el análisis solicitado:

    IND_ALUMNOS_GRUPO_REAL: Número real de alumnos por grupo.
    CAPACIDAD: Capacidad del aula asignada.
    ID_FECHA_GRUPO: Fecha de la clase.
    ID_HORA_INICIO y ID_HORA_FIN: Hora de inicio y fin de la clase.

Procederé a:

    Calcular la ocupación: Comparar el número de alumnos reales con la capacidad de las aulas para obtener un porcentaje de ocupación.
    Analizar patrones temporales: Verificar los días y franjas horarias de mayor y menor ocupación para identificar patrones en el uso de espacios

In [2]:
import pandas as pd

input_path = '/kaggle/input/uab-the-hack-2024/'
data = pd.read_csv(input_path+'grupos_ubicacio_merged.csv')

data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_GRUPO                515 non-null    object 
 1   ID_FECHA_GRUPO          515 non-null    object 
 2   ID_HORA_INICIO          515 non-null    object 
 3   ID_HORA_FIN             515 non-null    object 
 4   ID_CURSO_ACADEMICO      515 non-null    int64  
 5   ID_ASIGNATURA           515 non-null    int64  
 6   ID_TIPO_DOCENCIA        515 non-null    int64  
 7   ID_COD_GRUPO            515 non-null    int64  
 8   ID_PERIODO_DOCENTE      515 non-null    int64  
 9   IND_ALUMNOS_GRUPO_PREV  515 non-null    int64  
 10  IND_ALUMNOS_GRUPO_REAL  515 non-null    int64  
 11  IND_HORAS_PREVISTAS     515 non-null    float64
 12  AULA_ASIGNADA           515 non-null    object 
 13  DS_UBICACIO             515 non-null    object 
 14  ID_EDIFICI              515 non-null    ob

(                  ID_GRUPO ID_FECHA_GRUPO ID_HORA_INICIO ID_HORA_FIN  \
 0   2024-0-115-102714-1-31     2025-03-10       11:30:00    13:30:00   
 1   2024-0-115-102690-1-31     2025-03-10       08:30:00    10:30:00   
 2   2024-0-115-106050-1-21     2025-03-10       10:30:00    11:30:00   
 3  2024-0-115-102757-1-450     2025-03-10       11:30:00    13:30:00   
 4   2024-0-115-106048-1-21     2025-03-10       13:30:00    14:30:00   
 
    ID_CURSO_ACADEMICO  ID_ASIGNATURA  ID_TIPO_DOCENCIA  ID_COD_GRUPO  \
 0                2024         102714                 1            31   
 1                2024         102690                 1            31   
 2                2024         106050                 1            21   
 3                2024         102757                 1           450   
 4                2024         106048                 1            21   
 
    ID_PERIODO_DOCENTE  IND_ALUMNOS_GRUPO_PREV  IND_ALUMNOS_GRUPO_REAL  \
 0                   1                     158

# Convert date and time columns to datetime format for analysis
data['ID_FECHA_GRUPO'] = pd.to_datetime(data['ID_FECHA_GRUPO'])
data['ID_HORA_INICIO'] = pd.to_datetime(data['ID_HORA_INICIO'], format='%H:%M:%S').dt.time
data['ID_HORA_FIN'] = pd.to_datetime(data['ID_HORA_FIN'], format='%H:%M:%S').dt.time

# Calculate occupancy rate: (Actual Students / Capacity) * 100
data['Ocupacion'] = (data['IND_ALUMNOS_GRUPO_REAL'] / data['CAPACIDAD']) * 100

# Aggregate data to identify occupancy patterns by day of the week and time slots
data['DayOfWeek'] = data['ID_FECHA_GRUPO'].dt.day_name()
data['TimeSlot'] = pd.cut(pd.to_datetime(data['ID_HORA_INICIO'].astype(str)).dt.hour, 
                          bins=[0, 8, 12, 16, 20, 24], 
                          labels=["Night", "Morning", "Afternoon", "Evening", "Late Night"], 
                          right=False)

# Summary of occupancy by day of the week and time slot
occupancy_by_day_time = data.groupby(['DayOfWeek', 'TimeSlot']).Ocupacion.mean().unstack()

# Display the results to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Occupancy Analysis by Day and Time Slot", dataframe=occupancy_by_day_time)

occupancy_by_day_time

In [3]:
# Convert date and time columns to datetime format for analysis
data['ID_FECHA_GRUPO'] = pd.to_datetime(data['ID_FECHA_GRUPO'])
data['ID_HORA_INICIO'] = pd.to_datetime(data['ID_HORA_INICIO'], format='%H:%M:%S').dt.time
data['ID_HORA_FIN'] = pd.to_datetime(data['ID_HORA_FIN'], format='%H:%M:%S').dt.time

# Calculate occupancy rate: (Actual Students / Capacity) * 100
data['Ocupacion'] = (data['IND_ALUMNOS_GRUPO_REAL'] / data['CAPACIDAD']) * 100

# Aggregate data to identify occupancy patterns by day of the week and time slots
data['DayOfWeek'] = data['ID_FECHA_GRUPO'].dt.day_name()
data['TimeSlot'] = pd.cut(pd.to_datetime(data['ID_HORA_INICIO'].astype(str)).dt.hour, 
                          bins=[0, 8, 12, 16, 20, 24], 
                          labels=["Night", "Morning", "Afternoon", "Evening", "Late Night"], 
                          right=False)

# Summary of occupancy by day of the week and time slot
occupancy_by_day_time = data.groupby(['DayOfWeek', 'TimeSlot']).Ocupacion.mean().unstack()

# Display the results to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Occupancy Analysis by Day and Time Slot", dataframe=occupancy_by_day_time)

occupancy_by_day_time

/tmp/ipykernel_30/2330232462.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['TimeSlot'] = pd.cut(pd.to_datetime(data['ID_HORA_INICIO'].astype(str)).dt.hour,
/tmp/ipykernel_30/2330232462.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  occupancy_by_day_time = data.groupby(['DayOfWeek', 'TimeSlot']).Ocupacion.mean().unstack()


ModuleNotFoundError: No module named 'ace_tools'